# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv = "weatherData.csv"
weather_df = pd.read_csv(weather_csv)
weather_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,73.22,67,18,11.61,PF,1655096985
1,kapaa,22.0752,-159.3190,82.38,87,75,13.80,US,1655096986
2,sarai alamgir,32.9012,73.7559,102.83,19,80,6.22,PK,1655096986
3,broome,42.2506,-75.8330,64.62,89,75,5.75,US,1655096986
4,saskylakh,71.9167,114.0833,70.61,26,1,7.02,RU,1655096840


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
humidity = weather_df["Humidity"].astype(float)
locations = weather_df[["Latitude","Longitude"]].astype(float)

fig = gmaps.figure()
heat_layer = gmaps. heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_weather_df  = weather_df
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] > 70) & (ideal_weather_df["Max Temp"]) < 80]
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,kaunas,54.9000,23.9000,59.88,72,0,3.44,LT,1655096993
66,kamenka,51.3223,42.7678,65.21,62,0,6.85,RU,1655097009
68,paralimni,35.0375,33.9833,75.69,90,0,7.00,CY,1655097010
69,castro,-24.7911,-50.0119,36.82,74,0,3.31,BR,1655097010
71,homer,59.6425,-151.5483,62.49,60,0,8.99,US,1655097011


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_weather_df
params = {"radius": 5000,"type" :"restaurant","key":g_key}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url,params=params).json()
    results = response["results"]
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (IndexError):
        pass
hotel_df = hotel_df.dropna()

In [12]:
hotel_df.head(10)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,kaunas,54.9000,23.9000,59.88,72,0,3.44,LT,1655096993,Perkuno namai
68,paralimni,35.0375,33.9833,75.69,90,0,7.00,CY,1655097010,Amore Hotel Apartments
69,castro,-24.7911,-50.0119,36.82,74,0,3.31,BR,1655097010,Kuk's Burger
71,homer,59.6425,-151.5483,62.49,60,0,8.99,US,1655097011,Beluga Lake Lodge
84,hami,42.8000,93.4500,97.45,4,0,3.69,CN,1655097016,Dongcheng Sichuan Food Restaurant
97,yulara,-25.2406,130.9889,67.28,27,0,3.74,AU,1655097020,Tali Wiru
122,weinan,34.5036,109.5089,92.93,19,0,4.90,CN,1655096815,Lanzhou Beef Noodles
129,mahon,39.8885,4.2658,68.56,94,0,4.61,ES,1655097029,Hotel Catalonia Mirador des Port
154,kodiak,57.7900,-152.4072,62.11,55,0,9.22,US,1655097037,Subway
165,mount isa,-20.7333,139.5000,73.17,16,0,9.22,AU,1655097040,Abacus Motel


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

In [18]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)',stroke_color='rgba(0, 0, 150, 0.4)', 
                                 scale=2, info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))